In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#Road
import urllib
import gzip
import json
import requests
import mysql.connector
import datetime
import threading, time
import numpy as np #python陣列的套件，命名為np
from datetime import datetime
from threading import Thread
from urllib.request import urlretrieve
import re #字串多重分割套件

def func():
    global tStart #開始爬蟲時間
    global tEnd   #結束爬蟲時間
    global timenow#當前時間
    
    print ("crawling web page")
    urlretrieve = "https://tcgbusfs.blob.core.windows.net/blobtisv/GetVD.xml" #交通局開放資料，xml格式的純文字內容
    data = requests.get(urlretrieve).text #解析網頁內容後將str存入變數
    data = json.dumps(data,ensure_ascii=False) #ensure_ascii=False，解決中文格式變亂碼問題
    data = data.replace('vd:','vd').replace('/','') #字串內符號替代以便進行分割，把 vd: 替換成 vd，所有 / 清除
    data = re.split('<vdSectionDataSet><vdSectionData><vdSectionId>|<vdSectionId><vdSectionName>|<vdSectionName><vdAvgSpd>|<vdAvgSpd><vdAvgOcc>|<vdAvgOcc><vdTotalVol>|<vdTotalVol><vdMOELevel>|<vdMOELevel><vdStartWgsX>|<vdStartWgsX><vdStartWgsY>|<vdStartWgsY><vdEndWgsX>|<vdEndWgsX><vdEndWgsY>|<vdEndWgsY><vdSectionData><vdSectionData><vdSectionId>',data)
    #使用re套件進行多重分割
    a1=1
    a2=2
    a3=3
    #存取list物件用

    timenow = time.localtime(time.time())
    print(timenow)
    a=np.array(time.localtime(time.time()))#將時間格式轉換成陣列
    date = datetime.now().strftime('%Y_%m_%d')#取出當前時間 年_月_日
    vdExchangeTime = datetime.now().strftime("%Y-%m-%d %H:%M:%S") #檔案中無時間註記，以系統時間替代
    
    if a[1]==10:#當時間來到10月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="road_10", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==11:#當時間來到11月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="road_11", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==12:#當時間來到12月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="road_12", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    else:
        print("db not found")
        pass

    if True:
        print("1")
        sql="CREATE TABLE IF NOT EXISTS " +date+ " ( `vdSectionId` char(8) NOT NULL, `vdExchangeTime` datetime NOT NULL, `vdSectionName` VARCHAR(20) CHARACTER SET utf8 NOT NULL, `vdAvgSpd` float(16) NOT NULL , `vdAvgSpdlevel` VARCHAR(4) CHARACTER SET utf8 NOT NULL ) ENGINE=InnoDB DEFAULT CHARSET=latin1 AUTO_INCREMENT=0;"
        #date以日期名稱命名table
        try:
            c.execute(sql) #創建table
        except Exception as e:
            print("Mysql Error %d: %s" % (e.args[0], e.args[1]))
            pass
        #雨量站
        for i in range(0,807):#8071->807 #跑list長度的迴圈次數，分割後的list有8071個物件，list中每十個物件為一個測速站所有參數，一次迴圈處理一個
            d={} #創建字典
            #把值寫入字典
            d['vdSectionId']=data[a1]
            d['vdExchangeTime']=vdExchangeTime
            d['vdSectionName']=data[a2]
            d['vdAvgSpd']=data[a3]
            #道路速率評級寫入變數
            vdAvgSpdlevel = float(data[a3])
            if vdAvgSpdlevel>=0.00000 and vdAvgSpdlevel < 40.00000 :
                vdAvgSpdlevel = str(vdAvgSpdlevel)
                vdAvgSpdlevel = '塞車'
            elif vdAvgSpdlevel >= 40.00000 and vdAvgSpdlevel < 150.00000 :
                vdAvgSpdlevel = str(vdAvgSpdlevel)
                vdAvgSpdlevel = '順暢'
            else:
                vdAvgSpdlevel = str(vdAvgSpdlevel)
                vdAvgSpdlevel = '無資料'
            d["vdAvgSpdlevel"]=vdAvgSpdlevel
            #將字典的值放入變數
            vdSectionId = d["vdSectionId"]
            vdExchangeTime = d["vdExchangeTime"]
            vdSectionName = d["vdSectionName"]
            vdAvgSpd = d["vdAvgSpd"]
            vdAvgSpdlevel = d["vdAvgSpdlevel"]
            #print(d)
                
            sql = "INSERT INTO " + date + " (vdSectionId,vdExchangeTime,vdSectionName,vdAvgSpd,vdAvgSpdlevel) VALUES(%s,%s,%s,%s,%s)"
            #資料寫入table
            try:
                c.execute(sql,(vdSectionId,vdExchangeTime,vdSectionName,vdAvgSpd,vdAvgSpdlevel))
            except Exception as e:
                print("Mysql Error %d: %s" % (e.args[0], e.args[1]))
                pass
            a1=a1+10
            a2=a2+10
            a3=a3+10
            #處理完一個測速站跑下一個，直到跑完全部的
        conn.commit()
        conn.close()
    
def start():
    t = Thread(target=func) #跑func這個function
    t.start()
   
if __name__ == '__main__':
    counter=0
    a=5
    tStart1 = time.time()
    start() #跑start這個function
    tEnd1 = time.time()
    time.sleep(100-(tEnd1 - tStart1)) #設定每100秒起來檢查一次
    tEnd1 = time.time()
    tstart0=tStart1
    
    while(a>0):
        tStart1 = tEnd1
        
        if((tStart1-tstart0)>=300): #如果到300秒起來爬資料(原始資料更新頻率)
            counter=counter+1
        else:
            tstart0=tEnd1
            counter=counter+1
            
        if(counter%3==0):
            start()
        else:
            print("")    
            
        tEnd1 = time.time()
        time.sleep(100-(tEnd1 - tStart1))
        tEnd1 = time.time()
    